In [ ]:
from os import path

# Third-party
from astropy.constants import G
from astropy.io import fits, ascii
from astropy.stats import median_absolute_deviation
from astropy.table import Table, QTable, join
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import numpy as np
%matplotlib inline
from tqdm import tqdm

from twoface.db import (db_connect, AllStar, AllVisit, AllVisitToAllStar, NessRG,
                        StarResult, Status, JokerRun)
from twoface.plot import plot_two_panel, plot_phase_fold
from twoface.mass import get_m2_min, mf, period_at_surface, asini, stellar_radius
from twoface.log import log as logger
logger.setLevel(100)

import mesa_reader as mr
from helpers import tcirc_inv, compute_dlne, MESAHelper

In [ ]:
plot_path = '../paper/figures/'
# table_path = '../data/'
table_path = '../../twoface/paper/1-catalog/tables/'

In [ ]:
unimodal = QTable.read(path.join(table_path, 'highK-unimodal.fits'), character_as_bytes=False)
clean_uni = unimodal[(unimodal['clean_flag'] == 0)]
high_logg = clean_uni[clean_uni['LOGG'] > 2]
print('{0} unimodal, {1} clean, {2} high logg'
      .format(len(unimodal), len(clean_uni), len(high_logg)))

In [ ]:
P_surf = period_at_surface(1.36*u.Msun, high_logg['LOGG'], 
                           high_logg['e'], 0.5*u.Msun)
P_ratio = high_logg['P'] / P_surf

In [ ]:
broad_mask = np.array(['BROAD_LINES' in x for x in high_logg['STARFLAGS']])
broad_mask.sum()

In [ ]:
mask = (P_ratio < 6.) & (high_logg['e'] > 0.1)
mask.sum()

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.scatter(P_ratio, high_logg['e'])
ax.scatter(P_ratio[mask], high_logg['e'][mask])
ax.scatter(P_ratio[broad_mask], high_logg['e'][broad_mask])
ax.set_ylim(-0.05, 1)
ax.set_xlim(0.7, 1000)
ax.set_xscale('log')

In [ ]:
high_logg['APOGEE_ID', 'LOGG', 'P', 'e', 'TARGFLAGS'][mask]

In [ ]:
high_logg['APOGEE_ID', 'LOGG', 'TEFF', 'P', 'e'][mask & (~broad_mask)]